# A file containing all my loop functions

In [4]:
# import my own functions:
import sys
import ipynb
import re
sys.path.append('/project/Xelix_Project/utils')

from ipynb.fs.full.Regex_html_Functions import *
import logging
from unidecode import unidecode

In [3]:

def email_splitting(df):
    # grow lists iteratively, then create dataframe at the end
    iter_count_list = []
    email_id = []
    subject = []
    internal_date = []
    from_address = []
    from_name = []
    sent_to = []
    sent_to_how_many = []
    bcc = []
    cc = []
    reply_to_address = []
    attachments = []
    category_id = []
    category_slug = []
    has_attachment = []
    email_text_body = []
    initial_domain = []

    # new columns for splitting the emails
    is_part_of_thread = []
    num_in_thread = []
    thread_id = []

    # sort code related
    has_sort_code = []
    sort_code = []

    # language related
    mentions_bank = []
    mentions_bank_account = []
    mentions_change = []
    mentions_account_specifics = []

    # in the subject
    subject_mentions_bank = []
    subject_mentions_change = []
    
    # in the attachment
    attachment_mentions_bank = []

    # additional features
    mentions_statement = []
    mentions_invoice = []
    mentions_fraud = []
    has_attention_ext_flag = []
    has_alert_attach_flag = []

    # log error for normal emails too
    has_error = []

    # error emails:
    error_mail_id = []
    error_mail_body = []
    error_type = []

    ############################
    # splitting emails by this: # Disclaimer: need to check this is the same across languages etc
    split_string1 = "From: "
    split_string2 = "wrote: "

    # some regex strings I use
    find_emails_in_section = r'(?:<|\[)(?:mailto:)?(\b\w+@\w+\.\w+\b)(?:>|\])'
    find_all_hyperlinks = r'<(.*?)>'
    clean_slashes = r"\\(\s+)?"

    iter_count = 0
    error_count = 0

    # change name of df here
    for line in df.itertuples():

        iter_count += 1
    #     print(f'iter count {iter_count}, id: {line.id}')

        ################### if single email, we just append as we would
        if split_string1 not in line.body and split_string2 not in line.body:

            # save thread details
            iter_count_list.append(iter_count)
            is_part_of_thread.append(False)
            num_in_thread.append(False)
            thread_id.append(False)

            # save data
            email_id.append(line.id)
            attachments.append(line.attachments)
            category_id.append(line.category_id)
            category_slug.append(line.category_slug)
            has_attachment.append(line.has_attachment)
            first_domain = extract_domain_from_address(line.from_address)
            initial_domain.append(first_domain)

            # email specific
            subject.append(line.subject)
            internal_date.append(line.internal_date)
            from_address.append(line.from_address)
            from_name.append(line.from_address) # just save sender email instead here
            sent_to.append(line.to)
            sent_to_how_many.append(line.to.count(',')+1)
            bcc.append(line.bcc)
            cc.append(line.cc)
            reply_to_address.append(line.reply_to)

    #         # remove slashes
    #         this_email = re.sub(clean_slashes1," ",line.body)
    # #         this_email = re.sub(clean_slashes2," ",line.body)
    # #         this_email = re.sub(clean_slashes3," ",line.body)
    #         this_email = this_email.replace("  "," ")

            # conditionals
            sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
            has_sort_code.append(sort_code_bool)
            sort_code.append(sort_code_num)

            # language related
            mentions_bank.append(find_bank_mention_in_email(line.body))
            mentions_bank_account.append(find_bank_account_mention_in_email(line.body))
            mentions_change.append(find_update_words_in_email(line.body))
            mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))

            # in the subject
            subject_mentions_bank.append(find_bank_mention_in_email(line.subject))
            subject_mentions_change.append(find_update_words_in_email(line.subject))
            
            # in the attachment
            attachment_mentions_bank.append(find_bank_mention_in_email(line.attachments))

            # extra features
            mentions_statement.append(find_statement_mention_in_email(line.body))
            mentions_invoice.append(find_invoice_mention_in_email(line.body))
            mentions_fraud.append(find_cybercrime_fraud_mention_in_email(line.body))
            has_attention_ext_flag.append(find_attention_external_mention_in_email(line.body))
            has_alert_attach_flag.append(find_alert_attach_mention_in_email(line.body))

            # clean email body
            this_email = re.sub(clean_slashes," ",line.body)
            this_email = this_email.replace("  "," ")
            email_text_body.append(clean_text_re_links_brackets(unidecode(this_email)))
            has_error.append(False)

        ################### thread of emails
        else:
            if split_string1 in line.body:
                split_emails = line.body.split(split_string1)
            elif split_string2 in line.body:
                split_emails = line.body.split(split_string2)
            # try / except handling here
            try:
                for i, row in enumerate(split_emails):

                    iter_count_list.append(iter_count)
                    row = row.replace('\\n',' ') # replace this so we can search for strings

                     # save thread details
                    is_part_of_thread.append(True)
                    num_in_thread.append(int(len(split_emails)-(i)))
                    thread_id.append(str(line.id)+'_t')

                    # save regardless of thread or not
                    email_id.append(line.id)
                    attachments.append(line.attachments)
                    category_id.append(line.category_id)
                    category_slug.append(line.category_slug)
                    has_attachment.append(line.has_attachment)
                    initial_domain.append(extract_domain_from_address(line.from_address))

                    bcc.append(None)
                    cc.append(None)
                    reply_to_address.append(line.reply_to) # just keep this consistent?

                    # for every split email which wasn't the first
                    if i > 0:

                        # log who sent this email in thread
                        before_sent = row.split("Sent: ")[0]
                        this_email = re.findall(find_emails_in_section, before_sent)
                        row = re.sub(before_sent,"",row)

                        if not(this_email):
                            this_person = before_sent
                            from_address.append(this_person) # save output

                        else:
                            this_person = re.sub(find_all_hyperlinks,"",before_sent) 
                            from_address.append(this_email) # save output

                        from_name.append(this_person) # save output  

                        # log when this email in thread was sent
                        date_text, row = extract_text_between_start_end('Sent: ','To: ',None, row)
                        internal_date.append(date_text) # save output
    #                     print(f'iter: {iter_count} email_id: {line.id}, {date_text}')

                        # log who this email was sent to
                        to_text, row = extract_text_between_start_end('To: ','Subject: ',None, row)
                        to_emails = re.findall(find_all_hyperlinks, to_text)
                        sent_to.append(to_emails) # save output
                        sent_to_how_many.append(len(to_emails)) # save output

                        # log the subject of this email
                        this_subject, row = extract_text_between_start_end('Subject: ','      ','  ', row)
                        subject.append(this_subject) # save output

                        # additional cleaning if needed                 
                        row = row.replace("Sent: ","").replace("Sent:","")
                        row = row.replace("To: ","").replace("To:","")
                        row = row.replace("Subject: ","").replace("Subject:","")
                        row = row.replace(this_subject,"")

                        # conditionals
                        sort_code_bool, sort_code_num = find_sort_code_in_email(row)
                        has_sort_code.append(sort_code_bool)
                        sort_code.append(sort_code_num)

                        # language related
                        mentions_bank.append(find_bank_mention_in_email(row))
                        mentions_bank_account.append(find_bank_account_mention_in_email(row))
                        mentions_change.append(find_update_words_in_email(row))
                        mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

                        # in the subject
                        subject_mentions_bank.append(find_bank_mention_in_email(this_subject))
                        subject_mentions_change.append(find_update_words_in_email(this_subject))
                        
                        # in the attachment
                        attachment_mentions_bank.append(None)

                        # extra features
                        mentions_statement.append(find_statement_mention_in_email(row))
                        mentions_invoice.append(find_invoice_mention_in_email(row))
                        mentions_fraud.append(find_cybercrime_fraud_mention_in_email(row))
                        has_attention_ext_flag.append(find_attention_external_mention_in_email(row))
                        has_alert_attach_flag.append(find_alert_attach_mention_in_email(row))

                        # remove slashes
                        row = re.sub(clean_slashes," ",row)
    #                     row = re.sub(clean_slashes2," ",row)
    #                     row = re.sub(clean_slashes3," ",row)
                        row = row.replace("  "," ")

                        # clean email body
                        email_text_body.append(clean_text_re_links_brackets(unidecode(row)))
                        has_error.append(False) # ran without error

                    elif i == 0:

                        # first email in thread, save as if saving from the main data
                        subject.append(line.subject)
                        internal_date.append(line.internal_date)
                        from_address.append(line.from_address)
                        from_name.append(line.from_address) # just save sender email instead here
                        sent_to.append(line.to)
                        sent_to_how_many.append(line.to.count(',')+1)

                        # conditionals
                        sort_code_bool, sort_code_num = find_sort_code_in_email(row)
                        has_sort_code.append(sort_code_bool)
                        sort_code.append(sort_code_num)
                    
                        # language related
                        mentions_bank.append(find_bank_mention_in_email(row))
                        mentions_bank_account.append(find_bank_account_mention_in_email(row))
                        mentions_change.append(find_update_words_in_email(row))
                        mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))

                        # in the subject
                        subject_mentions_bank.append(find_bank_mention_in_email(line.subject))
                        subject_mentions_change.append(find_update_words_in_email(line.subject))
                        
                        # in the attachment
                        attachment_mentions_bank.append(find_bank_mention_in_email(line.attachments))

                        # extra features
                        mentions_statement.append(find_statement_mention_in_email(row))
                        mentions_invoice.append(find_invoice_mention_in_email(row))
                        mentions_fraud.append(find_cybercrime_fraud_mention_in_email(row))
                        has_attention_ext_flag.append(find_attention_external_mention_in_email(row))
                        has_alert_attach_flag.append(find_alert_attach_mention_in_email(row))

                        # remove slashes
                        row = re.sub(clean_slashes," ",row)
    #                     row = re.sub(clean_slashes2," ",row)
    #                     row = re.sub(clean_slashes3," ",row)
                        row = row.replace("  "," ")

                        # clean email body
                        email_text_body.append(clean_text_re_links_brackets(unidecode(row)))
                        has_error.append(False) # ran without error

            # email threw an error
            except re.error as e:

                error_count += 1
                has_error.append(True) # ran with error

                # need to count if all lists are same length - error could have happened somewhere
                arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
                         bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
                         email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
                         mentions_bank,mentions_change,mentions_account_specifics,has_error,
                         mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,
                         has_alert_attach_flag,initial_domain,mentions_bank_account,subject_mentions_bank,
                 subject_mentions_change,attachment_mentions_bank]

                # add None's to emails with an error - this could probably be smarter
                [iter_count_list,email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
                 bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
                 email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
                 mentions_bank,mentions_change,mentions_account_specifics,has_error,
                 mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,
                 has_alert_attach_flag,initial_domain,mentions_bank_account,subject_mentions_bank,
                 subject_mentions_change,attachment_mentions_bank
                ] = check_array_lengths_add_Nones(arrays)

                # log details for error emails
                error_mail_id.append(line.id)
                error_mail_body.append(line.body) 
                error_type.append(e)

    #             ################## if wanted ####################
    #             # print out the error emails only
    #             print("#" * 70, "\n\n")
    #             print(f'Regex error occurred: {e}\n\n')               
    #             print(f'email id: {line.id}, iter number: {iter_count}. So far there have been {error_count} errors.\n\n')
    #             print(f'subject line:\t{line.subject}')
    #             print(f'received from:\t{line.from_address}')
    #             print(f'sent to:\t{line.to}')
    #             #print(f'emails sent to: {line.to.count(',')+1} people')
    #             print(f'date sent:\t{line.internal_date}')
    #             print(f'slug:\t\t{line.category_slug}')
    #             print("")
    #             print(line.body)
    #             print("\n")
    #             ##################################################

    print("-" * 50, "\n")
    print(f'there were {error_count} errors in total.\n')
    print("-" * 50)    
    
    dict_data = {'iter_count_list':iter_count_list,'email_id':email_id,'is_part_of_thread':is_part_of_thread,'num_in_thread':num_in_thread,'thread_id':thread_id,
             'initial_domain':initial_domain,'from_address':from_address,'from_name':from_name,'sent_to':sent_to,
             'sent_to_how_many':sent_to_how_many,
             'bcc':bcc,'cc':cc,'reply_to_address':reply_to_address,
             'internal_date':internal_date,'category_id':category_id,'category_slug':category_slug,
             'has_sort_code':has_sort_code,'sort_code':sort_code,'has_attachment':has_attachment,
             'mentions_bank':mentions_bank,'mentions_change':mentions_change,'mentions_account_specifics':mentions_account_specifics,
             'mentions_bank_account':mentions_bank_account,'subject_mentions_bank':subject_mentions_bank,
             'subject_mentions_change':subject_mentions_change,'attachment_mentions_bank':attachment_mentions_bank,
             'mentions_statement':mentions_statement,'mentions_invoice':mentions_invoice,'mentions_fraud':mentions_fraud,
             'has_attention_ext_flag':has_attention_ext_flag,'has_alert_attach_flag':has_alert_attach_flag,
             'subject':subject,'attachments':attachments,'email_text_body':email_text_body
            }

    # check dictionary items are same length
    check_dict_value_lengths(dict_data)
    
    error_dict_data = {'email_id':error_mail_id,'email_text_body':error_mail_body,'error_type':error_type}
    # check dictionary items are same length
    check_dict_value_lengths(error_dict_data)
    
    new_df = pd.DataFrame.from_dict(dict_data)
    error_df = pd.DataFrame.from_dict(error_dict_data)
    
    # turns numeric columns into integers, replaces missing with 0
    new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']] = new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']].fillna(0).astype(int)
    
    return new_df, error_df

# Modelling loops

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display


# function for this loop
def sentence_by_sentence(df, model,save_prob=False):
    '''Classify the text of the emails in a sentence-by-sentence manner
    '''
    
    max_count = len(df)
    email_iter = 1

    if save_prob == True:
        sentence_cols = ['email_iter','email_id','num_in_thread',
                     'sentence','true_label','true_class','prob0','prob1','y_pred']
    else:
        sentence_cols = ['email_iter','email_id','num_in_thread',
                         'sentence','true_label','true_class','y_pred']
    
    df_list = []

    # to split the sentences by
    split_sentence_string = "[.?!]"
    
    f = IntProgress(min = 0, max = max_count)
    display(f)

    # change name of df here
    for email in df.itertuples():
        
        # try/except handling here
        try:

            clean_text = clean_websites(email.text)
            clean_text2 = clean_fluff(clean_text)
            sentences = re.split(split_sentence_string,clean_text2)

            # some counters
            email_iter += 1
            f.value = email_iter
            sum_score = 0

            len_email = len(sentences)

            for s in sentences:

                # check if this bit of the list is empty
                if len(s.strip()) == 0:
                    continue
                else:

#                     preds = model([s])
#                     this_label = preds.item()
#                     sum_score += this_label
                    
                    # display probabilities of each class
                    if save_prob == True:
                        proba = model.predict_proba([s])
                        probs = proba.tolist()
                    
                        prob0 = probs[0][0]
                        prob1 = probs[0][1]
                    
                        if prob0 > prob1:
                            this_label = 0
                        elif prob1 >= prob0:
                            this_label = 1
                        
                        sum_score += this_label
                        
                        # save row data
                        row_data = [email_iter, email.email_id, email.num_in_thread,
                                    s, email.label, email.label_text, prob0, prob1, this_label]
                        
                    else:

                        preds = model([s])
                        this_label = preds.item()
                        sum_score += this_label
                        
                        # save row data
                        row_data = [email_iter, email.email_id, email.num_in_thread,
                                    s, email.label, email.label_text, this_label]

                    df_row = pd.DataFrame([row_data],columns = sentence_cols)
                    df_list.append(df_row)

            ### save email sentence info
            sentence_df = pd.concat(df_list, ignore_index=True)

            ### save the meta email info
            perc_flag = "{:.1f}".format((sum_score*100)/len_email)

            # binary coding 
            y_pred = 1 if sum_score > 0 else 0
            diff = 0 if y_pred == email.label else 1
            
            # values to append to the big data frame
            df.at[email.Index,'email_iter'] = email_iter
            df.at[email.Index,'sum_score'] = sum_score
            df.at[email.Index,'len_email'] = len_email
            df.at[email.Index,'perc_flag'] = perc_flag
            df.at[email.Index,'y_pred'] = y_pred
            df.at[email.Index,'diff'] = diff
            
        except:
            print(f'error occurred at email_iter: {email_iter} email_id: {email.email_id} thread_id: {email.thread_id}')
#             logging.exception('')

    return sentence_df, df